In [132]:
import dateparser
import requests
import time

from functools import cached_property

class Api:
    _UNITS = dict(m=60, h=3_600, D=86_400, M=2_592_000)
    _MAX_LIMIT = 10_000
    _CANDLES_URL = 'https://api.bitfinex.com/v2/candles/trade:{}:t{}/hist?limit={}&start={}&end={}&sort=-1'

    @staticmethod
    def split_pair(pair):
        if len(pair) == 6:
            return (pair[:3], pair[3:])
        return tuple(pair.split(':'))
    
    @staticmethod
    def make_pair(symbol1, symbol2):
        if len(symbol1) == 3 and len(symbol2) == 3:
            return symbol1 + symbol2
        return symbol1 + ':' + symbol2

    @classmethod
    def interval_to_ms(cls, interval):
        unit = interval[-1]
        value = int(interval[0:-1])
        return value * cls._UNITS[unit] * 1_000

    @cached_property
    def pairs(self):
        return tuple(requests.get('https://api-pub.bitfinex.com/v2/conf/pub:list:pair:exchange').json()[0])

    @cached_property
    def pair_map(self):
        data = {}
        for p in api.pairs:
            f, t = split_pair(p)
            d = data.setdefault(t, set()).add(f)

        return {k:tuple(sorted(v)) for k, v in data.items()}
    
    def ochl(self, *, symbol, interval=None, start=None, end=None):
        interval = interval or '1D'
        ts_start = dateparser.parse(start or '2013-01-01 UTC').timestamp() * 1_000
        ts_end = dateparser.parse(end or 'now').timestamp() * 1_000
        step = Api.interval_to_ms(interval)
        ts0 = ts_start
        while ts0 < ts_end:
            if ts0 > ts_start:
                time.sleep(1.1)
            limit = min((ts_end - ts0) // step, self._MAX_LIMIT)
            if not limit:
                break
            ts1 = ts0 + limit * step
            url = self._CANDLES_URL.format(interval, symbol, limit, ts0, ts1)
            yield from requests.get(url).json()
            ts0 = ts1
    
    def pd_ochl(self, *, symbol, interval=None, start=None, end=None):
        data = self.ochl(symbol=symbol, interval=interval, start=start, end=end)
        df = pd.DataFrame(data, columns='time open close high low volume'.split())
        df.drop_duplicates(inplace=True)
        df['time'] = pd.to_datetime(df['time'], unit='ms')
        df.set_index('time', inplace=True)
        df.sort_index(inplace=True)        
        return df

api = Api()

In [134]:
all_data = {s: api.pd_ochl(symbol=api.make_pair(s, 'USD')) for s in api.pair_map['USD']}

In [146]:
result = None
for sym, df in all_data.items():
    df = df['close']
    df.name = sym
    if result is None:
        result = df
    else:
        result = pd.merge(result, df, how='outer', left_index=True, right_index=True)

In [236]:
import json
with open('crypto.jsonl', 'w') as fp:
    for ts in result.index:
        record = dict(timestamp=str(ts))
        for symbol in all_data.keys():
            try:
                df = all_data[symbol].loc[ts]
                for label in df.index:
                    record.setdefault(label, {})[symbol] = df[label]
            except KeyError:
                pass
        json.dump(record, fp, separators=(',', ':'))
        fp.write('\n')

In [227]:
json.dump?

Signature:
json.dump(
    obj,
    fp,
    *,
    skipkeys=False,
    ensure_ascii=True,
    check_circular=True,
    allow_nan=True,
    cls=None,
    indent=None,
    separators=None,
    default=None,
    sort_keys=False,
    **kw,
)
Docstring:
Serialize ``obj`` as a JSON formatted stream to ``fp`` (a
``.write()``-supporting file-like object).

If ``skipkeys`` is true then ``dict`` keys that are not basic types
(``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
instead of raising a ``TypeError``.

If ``ensure_ascii`` is false, then the strings written to ``fp`` can
contain non-ASCII characters if they appear in strings contained in
``obj``. Otherwise, all such characters are escaped in JSON strings.

If ``check_circular`` is false, then the circular reference check
for container types will be skipped and a circular reference will
result in an ``OverflowError`` (or worse).

If ``allow_nan`` is false, then it will be a ``ValueError`` to
serialize out of range ``float`` 

In [235]:
print(str(pd.Timestamp.now()))

2021-02-08 21:08:45.446359


In [133]:
api.pd_ochl(symbol='DOGUSD')

,open,close,high,low,volume
time,,,,,
2020-07-10,3879.0,3631.1,4950.0,3418.0,121.857898
2020-07-11,3508.0,3842.6,3938.4,3505.5,23.726461
2020-07-12,3771.1,3654.5,3840.3,3611.7,2.229347
2020-07-13,3604.8,3160.6,3610.7,3070.3,80.779781
2020-07-14,3155.0,3314.0,3360.6,3134.2,18.455876
2020-07-15,3280.7,3108.0,3339.2,3108.0,18.983336
2020-07-16,3137.5,3022.9,3137.5,2917.9,155.647223
2020-07-17,3025.4,3061.0,3108.5,2937.7,11.802551
2020-07-18,3060.2,3500.0,3744.7,3037.6,14.989777


In [123]:
all_data

{'AAVE': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'ADA': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'ALG': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'AMP': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'ANT': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'AST': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'ATO': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'AVAX': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'AVT': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'B21X': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'BAL': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'BAND': Empty DataFrame
 Columns: [open, close, high, low, volume]
 Index: [],
 'BAT': Empty DataFrame
 Columns: [open, close, 